In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
x = df.iloc[:,3:13].values
y = df.iloc[:,13].values

In [9]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Encode the labels for the columns
label_x_1 = LabelEncoder()
x[:, 1] = label_x_1.fit_transform(x[:, 1])

label_x_2 = LabelEncoder()
x[:, 2] = label_x_2.fit_transform(x[:, 2])

# One-hot encoding using ColumnTransformer
one_hot = ColumnTransformer(
    transformers=[('onehot', OneHotEncoder(), [1])],  # Specify the columns to one-hot encode
    remainder='passthrough'  # Keep the rest of the columns as is
)

# Apply the transformation
x = one_hot.fit_transform(x)

# Convert to a dense array if it's sparse
if hasattr(x, "toarray"):
    x = x.toarray()
x = x[:,1:]

In [26]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size = 0.80,random_state = 0)

In [27]:
#feature scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [28]:
#building the ANN
import keras
from keras.models import Sequential
from keras.layers import Dense

# Initializing the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu', input_dim=12))

# Adding the second hidden layer
classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu'))

# Adding the output layer
classifier.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

C:\Users\Ayush.LAPTOP-77UT5OAS\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [29]:
#compiling the ANN 
#Applying the Stochastic Gradient Descent to ANN
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [30]:
#fitting ann to training set
classifier.fit(x_train,y_train,batch_size=5,epochs=200)

Epoch 1/200
1600/1600 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7940 - loss: 0.5237
Epoch 2/200
1600/1600 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7896 - loss: 0.4393
Epoch 3/200
1600/1600 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7961 - loss: 0.4339
Epoch 4/200
1600/1600 ━━━━━━━━━━━━━━━━━━━━ 2s 996us/step - accuracy: 0.8184 - loss: 0.4287
Epoch 5/200
1600/1600 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8316 - loss: 0.4104
Epoch 6/200
1600/1600 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8324 - loss: 0.4103
Epoch 7/200
1600/1600 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8404 - loss: 0.3992
Epoch 8/200
1600/1600 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8370 - loss: 0.4044
Epoch 9/200
1600/1600 ━━━━━━━━━━━━━━━━━━━━ 2s 993us/step - accuracy: 0.8344 - loss: 0.4106
Epoch 10/200
1600/1600 ━━━━━━━━━━━━━━━━━━━━ 2s 1000us/step - accuracy: 0.8326 - loss: 0.4107
Epoch 11/200
1600/1600 ━━━━━━━━━━━━━━━━━━━━ 2s 996us/step - accuracy: 0.8461 - loss: 0.3928
Epoch 12/2

In [33]:
ypred = classifier.predict(x_test)
ypred = (ypred > 0.5)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [35]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_pred=ypred,y_true=y_test)
print(cm)
accuracy = (cm[0][0] + cm[1][1]) / (cm[0][0]+cm[0][1]+cm[1][0]+cm[1][1])
print(accuracy)

[[1536   59]
 [ 244  161]]
0.8485
